<a href="https://colab.research.google.com/github/Gideonngige/App/blob/master/Random_Forest_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("/content/Prices for Food Crops_Commodities_2012_to_2015 (EDA_cleaned).csv")
print(df.head())
print(df.columns)

   Unnamed: 0 Produce_Variety   Commodity_Type       Unit  Volume_in_Kgs  \
0           0    Horticulture         Cabbages    Ext Bag            126   
1           1    Horticulture  Cooking Bananas  Med Bunch             22   
2           2    Horticulture     Ripe Bananas  Med Bunch             14   
3           3    Horticulture          Carrots    Ext Bag            138   
4           4    Horticulture         Tomatoes     Lg Box             64   

  Values_in_Ksh        Date  produce_variety  Type_of_Commodity  Package_Type  \
0    KES2205.00  01/01/2012                1                  5             2   
1     KES511.00  01/01/2012                1                 11             4   
2     KES616.00  01/01/2012                1                 34             4   
3    KES2833.00  01/01/2012                1                  7             2   
4    KES3411.00  01/01/2012                1                 38             3   

   package_weight(Kg)  Day  Month  Year   Price  
0     

In [ ]:
# remove useless column
df = df.drop(columns=["Unnamed: 0"])

# Convert date
df["Date"] = pd.to_datetime(df["Date"])
df["month"] = df["Date"].dt.month
df["year"] = df["Date"].dt.year


# Sort properly
df = df.sort_values(by=["Commodity_Type", "Produce_Variety", "Date"])

# Keep only needed columns
df = df[[
    "Commodity_Type",
    "Produce_Variety",
    "Volume_in_Kgs",
    "Price",
    "Date",
    "Month",
    "Year"
]]

print(df.head())

    Commodity_Type Produce_Variety  Volume_in_Kgs   Price       Date  Month  \
21         Avocado    Horticulture             90  2232.0 2012-01-01      1   
44         Avocado    Horticulture             90  2439.0 2012-02-01      2   
67         Avocado    Horticulture             90  2252.0 2012-03-01      3   
90         Avocado    Horticulture             90  2044.0 2012-04-01      4   
113        Avocado    Horticulture             90  1939.0 2012-05-01      5   

     Year  
21   2012  
44   2012  
67   2012  
90   2012  
113  2012  


In [ ]:
df["future_price"] = df.groupby(
    ["Commodity_Type", "Produce_Variety"]
)["Price"].shift(-1)

df["target"] = (df["future_price"] > df["Price"]).astype(int)

df["price_t_1"] = df.groupby(
    ["Commodity_Type", "Produce_Variety"]
)["Price"].shift(1)

df["price_t_2"] = df.groupby(
    ["Commodity_Type", "Produce_Variety"]
)["Price"].shift(2)

df["ma_3"] = df.groupby(
    ["Commodity_Type", "Produce_Variety"]
)["Price"].shift(1).rolling(3).mean()

df["ma_7"] = df.groupby(
    ["Commodity_Type", "Produce_Variety"]
)["Price"].shift(1).rolling(7).mean()


In [ ]:
# Create trend label(target)
# 1 = UP, 0 = DOWN
# df["trend"] = (df["Price"] > df["price_last_time"]).astype(int)

In [ ]:

# input features
features = [
    "Volume_in_Kgs",
    "price_t_1",
    "price_t_2",
    "ma_3",
    "ma_7",
    "Month"
]



# X = df[features]
# y = df["trend"]


In [ ]:
# Train test split(Time-based)
from sklearn.model_selection import train_test_split

df = df.sort_values("Date")

split = int(len(df) * 0.8)
train = df.iloc[:split]
test = df.iloc[split:]

X_train = train[features]
y_train = train["target"]
X_test = test[features]
y_test = test["target"]


In [ ]:
# Train random forest classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    random_state=42
)



model.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, min_samples_split=5, n_estimators=300,
                       random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy:  0.5414847161572053
              precision    recall  f1-score   support

           0       0.71      0.47      0.56       145
           1       0.42      0.67      0.52        84

    accuracy                           0.54       229
   macro avg       0.56      0.57      0.54       229
weighted avg       0.60      0.54      0.55       229



In [ ]:
import joblib

joblib.dump(model, "price_trend_model.pkl")
print("Model saved as price_trend_model.pkl")


Model saved as price_trend_model.pkl
